# Visualize car data with Brunel

The Brunel Visualization language makes it easy to build interactive charts and diagrams that you can deploy rapidly.    
This notebook contains the steps and code to get you started with visualizing data with Brunel.

This notebook runs on a `Default Python 3.5 Free` environment.

You will use data about cars to graph the relationships between various properties, for example, how horsepower affects gas consumption. The cars data set was used for the 1983 American Statistical Association Data Exposition. This data set was collected by Ernesto Ramos and David Donoho and obtained from StatLib.

## Table of contents

This notebook has the following sections:
1. [Load the data](#data_set)
1. [Visualize the data](#visualize)
1. [Modify the DataFrame to highlight specific data](#highlight)
1. [Summary and next steps](#summary)

<a id="data_set"></a>
## 1. Load the data 
The car data is a freely available data set on the **IBM Watson Studio** community.

1. Open the Data Pane on the right (using the `1001` icon on the top right)
1. Locate the `cars.csv` file. If not present yet, download the file from this URL [Car performance data]("https://apsportal.ibm.com/exchange/public/entry/view/c81e9be8daf6941023b9dc86f303053b")
1. Select the code cell below and delete its content, then from the File's menu on the right sidebar, select the `cars.csv` file and  `Insert to code/Insert Credentials`. This will insert the file credentials from Object Storage in a variable that should be named `credentials_1` (if not, rename it, as this is what the prepared code uses afterwards)

In [ ]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'MaYnkzWXtUvsjH2gCnz65VnWj3lOzXyw3xWAhlkv_uWZ',
    'IAM_SERVICE_ID': 'iam-ServiceId-7d40cc21-927f-4801-8c80-39c23a5fcfeb',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'studioworkshop-donotdelete-pr-byxa4eth0oxr3z',
    'FILE': 'cars.csv'
}


### Setting up the environment and loading the `cars.csv` file into a Pandas DataFrame
Run the next cell to import pandas and Brunel libraries, load the data into a pandas DataFrame, and display the first five rows of data:

In [ ]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

body = cos_client.get_object(Bucket=credentials_1['BUCKET'],Key=credentials_1['FILE'])['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

cars = pd.read_csv(body,usecols=['mpg','cylinders','engine','horsepower','weight','acceleration','year','origin','name'])

cars.head(6)

<a id="visualize"></a>
## 2. Visualizing the data
You will now create some charts and diagrams using Brunel graphics representation.

The basic format of each call to Brunel is simple. Whether the command is a single line or a set of lines, the commands are concatenated together and the result interpreted as one command.

Here are some of the rules for using Brunel that you'll need in this notebook:
 * __DataFrame__: Use the `data` command to specify the pandas DataFrame. 
 * __Chart type__: Use commands like `chord` and `treemap` to specify a chart type. If you don't specify a type, the default chart type is a scatterplot. 
 * __Chart definition__: Use the `x` and `y` commands to specify the data to include on the x-axis and the y-axis.
 * __Styling__: Use commands like `color`, `tooltip`, and `label` to control the styling of the graph.
 * __Size__: Use the `width` and `height` key-value pairs to specify the size of the graph. The key-value pairs must be preceded with two colons and separated with a comma, for example: `:: width=800, height=300`
 
See detailed documentation on the Brunel Visualization language at __[Introduction to Brunel](http://brunel.mybluemix.net/docs/)__.

### Scatter plots
Run the next cell to show the relationship between the miles per gallon and the horsepower of the vehicles in a scatter plot.   
The color identifies the geographical origin of the vehicles. 

In [ ]:
import brunel
%brunel data('cars') x(mpg) y(horsepower) color(origin) :: width=800, height=300

### The graph output by Brunel above shows the distribution of Horsepower vs Miles-per-Galon.   
* It's showing quite obviously the trend that the more horsepower, the less mpg.   
* The coloring by car origin tends to indicate that American cars (in blue) have globally lower mpg than european (in red) and japanese (in yellow) ones.

> You can put your cursor over the chart and scroll to zoom in and out using your mouse wheel or middle button.  
When you zoom in, you can pan across the chart by clicking and dragging. 

### Now run the next cell to show the relationship between the horsepower and the weight of the cars in a scatter plot.
* The color is based on the origin of the cars.
* The tooltips show the name of the cars. 

In [ ]:
%brunel data('cars') x(horsepower) y(weight) color(origin) tooltip(name) :: width=800, height=300

### Inspect the results
* This shows that of course heavier cars are equipped with more powerful motors
* Some vertical rows also stand out, showing that motors HP are following some standards 
* You can hover the dots to identify odd instances, for example the 225HP car weighting only 3000 kg on the far right is a Buick Estate Wagon. The Lightest cars of all is a Datsun 1200

### Chord plot
* Run the next cell to show a chord plot that correlates the origin and number of cars produced and sold per year.
* The x and y commands specify that the origin is mapped to the year of manufacture. 
* The size of the segments is based on the number of cars.
* The color is based on the origin of the cars.

In [ ]:
%brunel data('cars') x(origin) y(year) chord size(#count) color(origin) :: width=500, height=400

### Analysing chord data output graph
* This graph shows that in 77,78, 81 and 82, japanese cars have had their market share increase since their ribbon on the year-end becomes wider.
* The drop in '79 vs a spike in '80 would call for further investigations.

### Treemap
A treemap can show many dimensions as recursively divided rectangles.

* Run the next cell to show a treemap that groups vehicles by their origin, year of manufacture, and number of cylinders.
* The color indicates the average gas mileage of the vehicles in each block.
* The numbers in each block are the number of cylinders.
* The size of the blocks reflects the number of vehicles in the category.
* The tooltips show all the information.

In [ ]:
%brunel data('cars') treemap x(origin, year, cylinders) color(mpg) mean(mpg) size(#count) label(cylinders) label(' cyl') tooltip(#all):: width=900, height=600

The treemap aboves allows to explore market structure with mostly 40cylinders cars for Japanese and European cars

<a id="highlight"></a>
## 3. Modify the DataFrame to highlight specific data
You can modify or add to the DataFrame to show data in different ways.   

In the following example, you apply a function that extract the `make` of the car as the first word from its `name` column.

The function is applied to the `name` column to create a new `make` column that consists of the car names that contain either "Ford" or "Buick".

In [ ]:
cars['make']=cars.name.map(lambda x: x.split(' ',1)[0])

Run the next cell to create a scatter chart that plots gas mileage versus engine size, colored by car make.

The Brunel command is split into two chart definitions that are combined by the overlay operator (a plus sign).

Both chart definitions set the x-axis, the y-axis, and the color to the same values but set the style to different values:
* the first chart definition sets the style of the dots
* the second definition sets the style of the words in the legend.  

The last line of the command sets the width and height of the chart.

In [ ]:
%%brunel data('cars') x(engine) y(mpg) color(make) style('size:50%; fill:#eee') +
     x(engine) y(mpg) color(make) style('text {font-size:14; font-weight:bold; fill:darker}') 
     :: width=800, height=800

<a id="summary"></a>
## 4. Summary and next steps
You explored different types of charts and formatting and learned how you can use the pandas DataFrame to refine your charts. Try changing the formatting of these charts, or creating your own. 

For more information about the Brunel Visualization language, see __[Introduction to Brunel](http://brunel.mybluemix.net/docs/)__.

Also read the Brunel blog at __[Working Vis - Perspectives on Actionable Visualization](http://brunelvis.org/)__.

### Authors

Dan Rope and Graham Wills are visualization architects at IBM. They created the Brunel visualization language.

Copyright © 2017-2018 IBM. This notebook and its source code are released under the terms of the MIT License.